In [1]:
import wx
import cv2
from PIL import Image
import imutils
import numpy as np
import time


In [2]:
class Frame(wx.Frame):
    def __init__(self, parent, ID=-1, title="CCP CV Assignment"):
        super().__init__(parent, ID, title)

        # Main window holds all the content
        self.main_window = wx.FlexGridSizer(rows=2, cols=1, hgap=10, vgap=10)

        # Contraollers holds all buttons and controls
        self.controllers = wx.FlexGridSizer(rows=4, cols=4, hgap=2, vgap=2)

        # Image Viewer
        self.img_viewer = wx.StaticBitmap(self)
        
        # Add image viewer and controllers to main window
        self.main_window.Add(self.img_viewer,0,0)
        self.main_window.Add(self.controllers,0,0)
        
        self.img_viewer.SetMinSize((500,500))
        self.controllers.SetMinSize((500,200))

        # Controller Buttons
        btn_load_img = wx.Button(self, label='Load Image')
        btn_load_img.Bind(wx.EVT_BUTTON, self.OnBrowse)

        btn_reset = wx.Button(self, label='Reset Image')
        btn_reset.Bind(wx.EVT_BUTTON, self.OnReset)

        btn_resize = wx.Button(self, label='Fit Image')
        btn_resize.Bind(wx.EVT_BUTTON, 
                        lambda event:self.OnResize(event,width=500))

        btn_crop = wx.Button(self, label='Crop 100')
        btn_crop.Bind(wx.EVT_BUTTON, self.OnCrop)

        btn_to_bw = wx.Button(self, label='B & W')
        btn_to_bw.Bind(wx.EVT_BUTTON, self.OnGray)

        btn_blur = wx.Button(self, label='Blur')
        btn_blur.Bind(wx.EVT_BUTTON, self.OnBlur)

        btn_edge = wx.Button(self, label='Detect Edge')
        btn_edge.Bind(wx.EVT_BUTTON, self.OnDetectEdge)

        btn_invert = wx.Button(self, label='Invert')
        btn_invert.Bind(wx.EVT_BUTTON, self.OnInvert)

        btn_detection = wx.Button(self, label='Detect Objects')
        btn_detection.Bind(wx.EVT_BUTTON, self.OnDetect)

        btn_on_cam = wx.Button(self, label='Turn On Camera')
        btn_on_cam.Bind(wx.EVT_BUTTON, self.OnCam)

        btn_release_cam = wx.Button(self, label='Release Camera')
        btn_release_cam.Bind(wx.EVT_BUTTON, self.OnCapture)

        btn_rotate = wx.Button(self, label='Rotate 10')
        btn_rotate.Bind(wx.EVT_BUTTON, self.OnRotate)

        btn_contour = wx.Button(self, label='Detect Contour')
        btn_contour.Bind(wx.EVT_BUTTON, self.OnContour)

        self.controllers.Add(btn_load_img, 0, 0)
        self.controllers.Add(btn_reset, 0, 0)
        self.controllers.Add(btn_resize, 0, 0)
        self.controllers.Add(btn_crop, 0, 0)
        self.controllers.Add(btn_to_bw, 0, 0)
        self.controllers.Add(btn_blur, 0, 0)
        self.controllers.Add(btn_edge, 0, 0)
        self.controllers.Add(btn_invert, 0, 0)
        self.controllers.Add(btn_detection, 0, 0)
        self.controllers.Add(btn_on_cam, 0, 0)
        self.controllers.Add(btn_release_cam, 0, 0)
        self.controllers.Add(btn_rotate, 0, 0)
        self.controllers.Add(btn_contour, 0, 0)
        
        self.SetSizer(self.main_window)
        self.Fit()
        
        self.InitYolo()

    def InitYolo(self):
        # Load YOLO model and COCO class names
        self.net = cv2.dnn.readNet('../yolov3/yolov3.weights', '../yolov3/yolov3.cfg')
        self.classes = []
        with open('../yolov3/coco.names', 'r') as f:
            self.classes = f.read().strip().split('\n')

    def OnDetect(self, event):
        # Prepare input image for YOLO model
        img = self.img_raw

        height, width = img.shape[:2]

        # img = img.astype(np.float32)
        
        blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        
        # Get detection results
        outs = self.net.forward(self.net.getUnconnectedOutLayersNames())

        # Process detection results
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = scores.argmax()
                confidence = scores[class_id]
                if confidence > 0.7:  # Set a confidence threshold
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    # Draw bounding box and class label
                    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 1)
                    label = f'{self.classes[class_id]}: {confidence:.2f}'
                    cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)

        self.ShowCV2Image(img)

    def OnContour(self, event):
        img = self.GetCurrentImage()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #Gaussian Blur
        blurred_image = cv2.GaussianBlur(img , (5,5) , 0)
    
        #Canny
        edges = cv2.Canny(img , 100 ,200)

        #Identify contours in edge detected images
        contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #print(contours)

        #Create a copy of the original image for drawing contours
        contour_image = img.copy()

        #Draw and display contours
        cv2.drawContours(contour_image , contours , -1 , (0,255,0) , 2)
        self.ShowCV2Image(contour_image)

    def OnBrowse(self, event):
        wildcard = 'PNG files (*.png)|*.png|BMP files (*.bmp)|*.bmp|JPEG files (*.jpg)|*.jpg|JPEG files (*.jpeg)|*.jpeg'
        openFileDialog = wx.FileDialog(self, "Open", "", "", wildcard,
                                       wx.FD_OPEN | wx.FD_FILE_MUST_EXIST)
        openFileDialog.ShowModal()

        # Use cv2 to open image
        file_path = openFileDialog.GetPath()
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        self.img_raw = img        

        self.ShowCV2Image(img)

    def OnResize(self, event, width=500):
        img = self.GetCurrentImage()
        # resized_img = imutils.resize(img, width=width)
        or_width = img.shape[1]
        or_height = img.shape[0]
        ratio = width / or_width
        height = int(or_height*ratio)
        
        resized_img = cv2.resize(img, (width, height))
        
        self.ShowCV2Image(resized_img)

    def OnCrop(self, event,x=0,y=0,w=100,h=100):
        '''
        Crop a 100 by 100-pixel image
        '''
        img = self.GetCurrentImage()
        cropped_img = img[y:y+h, x:x+w]
        self.ShowCV2Image(cropped_img)

    def OnReset(self, event):
        img = self.img_raw
        self.ShowCV2Image(img)

    def OnBlur(self, event):
        img = self.GetCurrentImage()
        img_blur = cv2.GaussianBlur(img, (21,21), 0)
        self.ShowCV2Image(img_blur)

    def OnGray(self, event):
        img = self.GetCurrentImage()
        gray_image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        self.ShowCV2Image(gray_image)

    def OnDetectEdge(self, event):
        img = self.GetCurrentImage()
        edge = cv2.Canny(img, 100, 200)
        self.ShowCV2Image(edge)

    def OnInvert(self, event):
        img = self.GetCurrentImage()
        inverted = 255 - img
        self.ShowCV2Image(inverted)

    def OnCam(self, event):
        self.release_cap = False
        
        self.cap = cv2.VideoCapture(0)
        # Read the frame
        ret1, frame1 = self.cap.read()
        prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

        n = 0
        while n < 30:
            time.sleep(1)
            ret, frame2 = self.cap.read()
            current_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
            
            # Calculate optical flow
            flow = cv2.calcOpticalFlowFarneback(prev_gray, current_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

            # Draw motion vectors
            motion_image = np.copy(frame2)
            for y in range(0, motion_image.shape[0], 10):
                for x in range(0, motion_image.shape[1], 10):
                    dx, dy = flow[y, x]
                    cv2.arrowedLine(motion_image, (x, y), (int(x+dx), int(y+dy)), (0, 0, 255), 1)
    
            prev_gray = current_gray

            print('Dwaing image')

            # self.ShowCV2Image(motion_image)
            cv2.imshow('Optical Flow', motion_image)
            
            # self.release_cap = True
            n = n + 1
            

            if self.release_cap:
                self.cap.release()
                self.release_cap = False
                cv2.destroyAllWindows()
                cv2.waitKey(1)
                break

    def OnRotate(self, event):
        img = self.GetCurrentImage()
        #Rotation
        angle = 10
        rows,cols = img.shape[:2]
        rotation_matrix = cv2.getRotationMatrix2D((cols/2 , rows/2) , angle , 1)
        rotated_matrix = cv2.warpAffine(img , rotation_matrix , (cols,rows))
        self.ShowCV2Image(rotated_matrix)
                
    def OnCapture(self, event):
        self.release_cap = True

    def GetCurrentImage(self):
        '''
        Get the current displaying image, and return as a cv image object
        '''
        img = self.img_viewer.GetBitmap().ConvertToImage()
        img_buffer = img.GetDataBuffer()
        
        pil_img = Image.frombuffer('RGB', (img.Width, img.Height), img_buffer)
        arr_img = np.array(pil_img)

        cv_img = cv2.cvtColor(arr_img, None)
        return cv_img

    
    def ShowCV2Image(self, img):
        '''
        Display a cv2 image object
        img: cv2 image object
        '''
        img_container = wx.Image(img.shape[1], img.shape[0])
        pil_img = Image.fromarray(img)
        img_container.SetData(pil_img.convert("RGB").tobytes())
        
        bmp_img = wx.Bitmap(img_container)
        self.img_viewer.SetBitmap(bmp_img)

app = wx.App()
frame = Frame(None)
frame.Show()
app.MainLoop()

2023-08-24 16:04:21.091 python3.10[32115:2268554] +[CATransaction synchronize] called within transaction
2023-08-24 16:04:21.164 python3.10[32115:2268554] +[CATransaction synchronize] called within transaction
2023-08-24 16:04:34.769 python3.10[32115:2268554] +[CATransaction synchronize] called within transaction
2023-08-24 16:04:34.825 python3.10[32115:2268554] +[CATransaction synchronize] called within transaction


0

In [3]:
import cv2 
import numpy as np

# Capture video feed 
cap = cv2.VideoCapture(0)

# Read the frame
ret1, frame1 = cap.read()
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

while cap.isOpened():
    ret, frame2 = cap.read()
    current_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, current_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Draw motion vectors
    motion_image = np.copy(frame2)
    for y in range(0, motion_image.shape[0], 10):
        for x in range(0, motion_image.shape[1], 10):
            dx, dy = flow[y, x]
            cv2.arrowedLine(motion_image, (x, y), (int(x+dx), int(y+dy)), (0, 0, 255), 1)
    
    prev_gray = current_gray

    # cv2.imshow('Video', frame1)
    # cv2.imshow('Gray Video', prev_gray)
    cv2.imshow('Optical Flow', motion_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


-1